In [5]:
!pip uninstall -y torch torchvision torchaudio triton transformers trl peft bitsandbytes accelerate datasets tokenizers pyarrow
print("✅ All packages removed")

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: triton 2.1.0
Uninstalling triton-2.1.0:
  Successfully uninstalled triton-2.1.0
Found existing installation: transformers 4.40.2
Uninstalling transformers-4.40.2:
  Successfully uninstalled transformers-4.40.2
Found existing installation: trl 0.9.4
Uninstalling trl-0.9.4:
  Successfully uninstalled trl-0.9.4
Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Successfully uninstalled peft-0.10.0
Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Found existing in

In [6]:
# Install PyTorch 2.1.2 (compatible with Triton 2.1.0 and BitsAndBytes)
!pip install -q torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121

# Install Triton 2.1.0
!pip install -q triton==2.1.0

# Install data/model libraries
!pip install -q pyarrow==19.0.1 datasets==2.20.0

# Install transformers + training libraries
!pip install -q transformers==4.40.2 tokenizers==0.19.1
!pip install -q trl==0.9.4 peft==0.10.0 accelerate==0.28.0 bitsandbytes==0.43.1

print("✅ All packages installed")
print("⚠️ RESTART SESSION NOW!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 484.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 6.7 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 22.3 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.18.0 requires transformers>=4.33.1, which is not installed.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtune 0.6.1 requires tokenizers, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0

In [7]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import sys
import torch
import triton
import transformers
import trl
import peft
import bitsandbytes

print("=" * 60)
print("ENVIRONMENT VERIFICATION")
print("=" * 60)
print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"Triton: {triton.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Transformers: {transformers.__version__}")
print(f"TRL: {trl.__version__}")
print(f"PEFT: {peft.__version__}")
print(f"BitsAndBytes: {bitsandbytes.__version__}")
print("=" * 60)

# Test critical imports
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset

print("✅ ALL IMPORTS SUCCESSFUL - NO ERRORS!")


ENVIRONMENT VERIFICATION
Python: 3.11.13
PyTorch: 2.1.2+cu121
Triton: 2.1.0
CUDA Available: True
Transformers: 4.40.2
TRL: 0.9.4
PEFT: 0.10.0
BitsAndBytes: 0.43.1


2025-12-01 14:54:08.959378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764600849.150224     288 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764600849.202607     288 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✅ ALL IMPORTS SUCCESSFUL - NO ERRORS!


In [ ]:
# === EDITABLE CONFIG ===
config = {
    "model_name": "microsoft/Phi-3-mini-4k-instruct",
    # common possible dataset locations: tries these in order
    "dataset_candidates": [
        "/kaggle/input/text-to-cypher-supply/text_to_cypher_10000.csv",
    ],
    "output_dir": "./phi3_qlora_output",
    "final_model_dir": "./phi3_qlora_final",
    "max_seq_length": 2048,   # kept for clarity; tokenization will handle truncation
    "train_test_split": 0.15,
    "num_epochs": 2,
    "batch_size": 2,
    "gradient_accumulation": 8,
    "learning_rate": 2e-4,
    "lora_r": 64,
    "lora_alpha": 32,
    "lora_dropout": 0.1,
    "seed": 42,
}
print("Config summary:")
for k,v in config.items():
    print(f"  {k}: {v}")


In [9]:
import os, time, math

In [10]:
def find_dataset(candidates):
    for p in candidates:
        if os.path.exists(p):
            return p
    return None

dataset_file = find_dataset(config["dataset_candidates"])
if dataset_file is None:
    print("No dataset found in common paths. Please upload your CSV and set `dataset_file` variable.")
    raise FileNotFoundError("Dataset CSV not found. Put it in /kaggle/input/ or current working dir.")
else:
    print("Using dataset:", dataset_file)
    config["dataset_file"] = dataset_file


Using dataset: /kaggle/input/text-to-cypher-supply/text_to_cypher_10000.csv


In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    bnb_4bit_use_double_quant=True,
)
print("BNB config prepared. compute dtype:", bnb_config.bnb_4bit_compute_dtype)


BNB config prepared. compute dtype: torch.float16


In [12]:
print("Loading model (this can take 30-90s depending on network/GPU)...")
t0 = time.time()

model = AutoModelForCausalLM.from_pretrained(
    config["model_name"],
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    low_cpu_mem_usage=True,
)

tokenizer = AutoTokenizer.from_pretrained(config["model_name"], trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Loaded model in {time.time()-t0:.1f}s. Device map:")
try:
    print(model.hf_device_map)
except Exception:
    pass


Loading model (this can take 30-90s depending on network/GPU)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model in 37.4s. Device map:
{'': 0}


In [13]:
print("Preparing model for k-bit training and attaching LoRA adapters...")
model = prepare_model_for_kbit_training(model)

lora_cfg = LoraConfig(
    r=config["lora_r"],
    lora_alpha=config["lora_alpha"],
    lora_dropout=config["lora_dropout"],
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_cfg)
print("LoRA attached. Trainable params:")
model.print_trainable_parameters()


Preparing model for k-bit training and attaching LoRA adapters...
LoRA attached. Trainable params:
trainable params: 35,651,584 || all params: 3,856,731,136 || trainable%: 0.9243989986031528


In [14]:
print("Loading dataset with Hugging Face datasets...")
ds = load_dataset("csv", data_files=config["dataset_file"])["train"]
print("Total rows in CSV:", len(ds))
print("Columns:", ds.column_names)
# Quick peek
print(ds[0])


Loading dataset with Hugging Face datasets...


Generating train split: 0 examples [00:00, ? examples/s]

Total rows in CSV: 10000
Columns: ['instruction', 'cypher']
{'instruction': 'Top 5 products by total ordered quantity.', 'cypher': 'MATCH (p:Product)<-[:ORDERS]-(o:CustomerOrder) WITH p, sum(o.quantity) AS qty RETURN p.name, qty ORDER BY qty DESC LIMIT 5;'}


In [15]:
print("Splitting dataset...")
ds = ds.train_test_split(test_size=config["train_test_split"], seed=config["seed"])
train_ds = ds["train"]
eval_ds = ds["test"]
print("Train size:", len(train_ds), "Eval size:", len(eval_ds))


Splitting dataset...
Train size: 8500 Eval size: 1500


In [16]:
# This assumes CSV has columns named 'instruction' and 'cypher'.
# If your columns have different names, change the keys below.

IN_COL = "instruction"
OUT_COL = "cypher"

def format_for_phi3(batch):
    texts = []
    for inst, cy in zip(batch[IN_COL], batch[OUT_COL]):
        # phi-3 chat-style template
        txt = f"<|user|>\n{inst}<|end|>\n<|assistant|>\n{cy}<|end|>"
        texts.append(txt)
    return {"text": texts}

# apply
train_ds = train_ds.map(format_for_phi3, batched=True, remove_columns=train_ds.column_names)
eval_ds = eval_ds.map(format_for_phi3, batched=True, remove_columns=eval_ds.column_names)

# small shuffle for training
train_ds = train_ds.shuffle(seed=config["seed"])
print("Sample formatted text:", train_ds[0]["text"][:240])


Map:   0%|          | 0/8500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Sample formatted text: <|user|>
Show orders delayed by more than 7 days.<|end|>
<|assistant|>
MATCH (o:CustomerOrder) WHERE o.delay > 7 RETURN o.id, o.delay ORDER BY o.delay DESC;<|end|>


In [20]:
training_args = TrainingArguments(
    output_dir=config["output_dir"],
    per_device_train_batch_size=config["batch_size"],
    per_device_eval_batch_size=config["batch_size"],
    gradient_accumulation_steps=config["gradient_accumulation"],
    num_train_epochs=config["num_epochs"],
    learning_rate=config["learning_rate"],
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_steps=50,
    weight_decay=0.01,

    # FIXED LINE ↓↓↓
    evaluation_strategy="steps",

    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    logging_steps=25,
    logging_first_step=True,
    report_to="none",
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
    gradient_checkpointing=True,
    load_best_model_at_end=True,
    seed=config["seed"],
)


In [21]:
def formatting_func(example):
    # SFTTrainer expects a function that returns text string(s)
    return example["text"]

# attach tokenizer on the model as a fallback (some older code expects model.tokenizer)
model.tokenizer = tokenizer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    formatting_func=formatting_func,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
print("SFTTrainer created.")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1910: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/8500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

SFTTrainer created.


/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [22]:
print("="*60)
print("STARTING TRAINING")
print("="*60)
t0 = time.time()
trainer.train()
print("Total training time (min):", (time.time() - t0)/60)


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


STARTING TRAINING


Step,Training Loss,Validation Loss
100,0.045000,0.042861
200,0.040600,0.043898
300,0.040600,0.041028
400,0.040400,0.040392
500,0.039800,0.040802
600,0.039300,0.039667
700,0.039700,0.038963
800,0.038600,0.039009
900,0.038100,0.038804
1000,0.039000,0.038696


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To main

Total training time (min): 199.99186536868413


In [23]:
print("Saving final model and tokenizer...")
trainer.model.save_pretrained(config["final_model_dir"])
tokenizer.save_pretrained(config["final_model_dir"])
print("Saved to:", config["final_model_dir"])
print("Contents:", os.listdir(config["final_model_dir"]))


Saving final model and tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Saved to: ./phi3_qlora_final
Contents: ['special_tokens_map.json', 'added_tokens.json', 'adapter_config.json', 'tokenizer.model', 'adapter_model.safetensors', 'tokenizer_config.json', 'tokenizer.json', 'README.md']


In [24]:
import shutil

save_path = config["final_model_dir"]        # example: "/kaggle/working/phi3_qlora_final"
zip_path = "/kaggle/working/phi3_qlora_final_zip"

# Create ZIP file
shutil.make_archive(zip_path, 'zip', save_path)

print("Zipped model saved at:", zip_path + ".zip")
print("You can now download this file from the right sidebar → Files")


Zipped model saved at: /kaggle/working/phi3_qlora_final_zip.zip
You can now download this file from the right sidebar → Files


In [25]:
import zipfile

zip_file = "phi3_qlora_final_zip.zip"
extract_to = "./phi3_qlora_final"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extracted to:", extract_to)


Extracted to: ./phi3_qlora_final


In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

bnb = BitsAndBytesConfig(load_in_4bit=True)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    quantization_config=bnb,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./phi3_qlora_final")

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "./phi3_qlora_final")

model.eval()


The repository for microsoft/Phi-3-mini-4k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3-mini-4k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for microsoft/Phi-3-mini-4k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3-mini-4k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
             

In [30]:
prompt = "<|user|>Top 5 products by total ordered quantity.\n<|end|>\n<|assistant|>\n"

tokens = tokenizer(prompt, return_tensors="pt").to(model.device)
output = model.generate(**tokens, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))


Top 5 products by total ordered quantity.
 MATCH (p:Product)<-[:ORDERS]-(o:CustomerOrder) WITH p, sum(o.quantity) AS qty RETURN p.name, qty ORDER BY qty DESC LIMIT 5;
